In [41]:
cd /home/rockwell/CameraWork/

/home/rockwell/CameraWork


In [42]:
ls

180208-30.mp4  180312-80gaussfilt9px.mp4  180312-82.bag
180213-55.mp4  180312-80medfilt5px.mp4    D415 parameter comparison.ods
180213-56.mp4  180312-80.mp4              DefaultPreset_D415.json
180312-80.bag  180312-81.bag


In [43]:
# Here is where you need to actually enter stuff
# the name of the file you want to process
filename='180312-81.bag'
# The background subtraction parameters (may change if the camera height (or other acquisition parameters) changes)
backgroundmin=0
backgroundmax=90
# the framerate you used to record (fps)
framerate=90
# You may only want to extract a certain number of frames (especially for large files)
# The default is set to 1000 because you may not have enough RAM to process more 
startframe=0
endframe=100

In [44]:
# imports. you may need to sudo apt-get install rosbag, rospy, cv-bridge (all ros packages) and some other stuff
import rosbag, rospy, numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation
import IPython.display
import cv2
import cv_bridge
import subprocess
import itertools
import os
from copy import copy,deepcopy

In [45]:
# simple command to pipe frames to an ffv1 file
def write_frame_preview(filename,frames=np.empty((0,)),threads=6,camera_fs=30,pixel_format='rgb24',codec='h264',
               slices=24,slicecrc=1,frame_size=None,depth_min=0,depth_max=80,get_cmd=False):
    
    if not frame_size and type(frames) is np.ndarray:
        frame_size='{0:d}x{1:d}'.format(frames.shape[2],frames.shape[1])
    elif not frame_size and type(frames) is tuple:
        frame_size='{0:d}x{1:d}'.format(frames[0],frames[1])
        
    command= [ 'ffmpeg',
          '-y',
          '-threads',str(threads),
          '-framerate',str(camera_fs),
          '-f','rawvideo',
          '-s',frame_size,
          '-pix_fmt',pixel_format,
          '-i','-',
          '-an',
          '-vcodec',codec,
          '-slices',str(slices),
          '-slicecrc',str(slicecrc),
          '-r',str(camera_fs),
          '-pix_fmt','yuv420p',
          filename ]
    
    if get_cmd:
        return command

    pipe=subprocess.Popen(command,stdin=subprocess.PIPE,stderr=subprocess.PIPE)
    
    # scale frames d00d

    cmap=plt.get_cmap('jet')
    
    for i in range(frames.shape[0]):
        disp_img=deepcopy(frames[i,:,:].astype('float32'))
        disp_img=(disp_img-depth_min)/(depth_max-depth_min)
        disp_img[disp_img<0]=0
        disp_img[disp_img>1]=1
        disp_img=np.delete(cmap(disp_img),3,2)*255
        pipe.stdin.write(disp_img.astype('uint8').tostring())
        
    pipe.stdin.close()
    pipe.wait()

In [46]:
bag=rosbag.Bag(filename,'r')

In [47]:
tmp=bag.get_type_and_topic_info()

In [48]:
iterator=bag.read_messages(topics='/device_0/sensor_0/Depth_0/image/data')

In [49]:
img=[]
bridge=cv_bridge.CvBridge()
for (topic,msg,t) in itertools.islice((iterator),startframe,endframe): #frame window you want to extract
    img.append(bridge.imgmsg_to_cv2(msg,'mono16'))
img=np.array(img)
#img.shape

In [50]:
# non-background subtracted video
save_file=os.path.splitext(os.path.basename(filename))[0]
# remove the hashtag in the line below to get non-background subtracted mp4
#write_frame_preview(save_file+'raw.mp4',img,depth_min=0,depth_max=1000,camera_fs=framerate)

In [51]:
# background subtraction
img=(np.median(img,0)-img).astype('uint16')
img[img<backgroundmin]=0
img[img>backgroundmax]=0

In [52]:
# mp4 for video with simple background subtraction. If this looks good you can go ahead and do whatever filter you want below
write_frame_preview(save_file+'.mp4',img,depth_min=backgroundmin,depth_max=backgroundmax,camera_fs=framerate)

In [53]:
# median filter
filt_img=deepcopy(img)
for i in range(img.shape[0]):
    filt_img[i,...]=cv2.medianBlur(img[i,...],1) 

In [54]:
write_frame_preview(save_file+'medfilt5px.mp4',filt_img,depth_min=backgroundmin,depth_max=backgroundmax,camera_fs=framerate)

In [55]:
# gaussian filter
filt_img=deepcopy(img)
for i in range(img.shape[0]):
    filt_img[i,...]=cv2.GaussianBlur(img[i,...],(1,1,),0) 
# the larger the ordered pair, the less of the mouse body I see. the last number doesnt seem to do anything

#### write_frame_preview(save_file+'gaussfilt9px.mp4',filt_img,depth_min=backgroundmin,depth_max=backgroundmax,camera_fs=framerate)